In [1]:
# Instalo geopandas

#conda install -c conda-forge geopandas 

In [2]:
# SP: Importo libreria Pandas para poder crear DataFrames. Y añado esta linea de codigo para poder ver todas filas.
# EN: Import Pandas library to create DataFrames. And added this line of code to be able to see all the rows.
import pandas as pd
pd.options.display.max_rows = None

# SP: Importo libreria Request para poder pedir datos a otros páginas.
# EN: I import the Request library to be able to request data from other pages.
import requests

# SP: Esta libreria es para crear el token que necesito para meterme en la segunda api, pero finalmente no la uso.
# EN: This library is to create the token that I need to get into the second API, but in the end I don't use it.
from dotenv import dotenv_values

# SP: Importo libreria Calculate_Distances para poder calcular la latitud y longitud de los centros deportivos
# EN: I import the Calculate_Distances library to be able to calculate the latitude and longitude of the sports centers
from shapely.geometry import Point
import geopandas as gpd

# SP: Importo numpy para realizar las operaciones matemáticas.
# EN: I import numpy to perform the math operations.
import numpy as np

In [3]:
# SP: LLamamos al archivo  bicimad_station.csv
# EN: We call the file bicimad_station.csv

'''
SP: Los conjuntos de datos contienen información de las estaciones de BiciMAD, incluida su ubicación (es decir, latitud/longitud).
Puede encontrar los archivos .csv en la carpeta de datos.

EN: The datasets contains information from the BiciMAD stations including their location (i.e.: latitude / longitude).
You may find the .csv files in the data folder.

'''

bicimad = pd.read_csv('data/bicimad_stations.csv', sep='\t')

bicimad.columns = [e.replace(' ', '_').lower() for e in bicimad.columns]

bicimad.head()

,unnamed:_0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"


In [4]:
'''
SP:
DESCRIPCIÓN DEL OBJETO ESTACION

id: Código de la Estación Base
latitude: Latitud de la estación en formato WGS84
longitude: Longitud de la Estación en formato WGS84
name: Nombre de la Estación
light: Grado de Ocupación (0=baja, 1=media, 2=alta, 3=inactiva)
number: Denominación lógica de la Estación Base
actívate: Estación activa (0=No activa, 1=activa)
no_available: Disponibilidad de la Estación (0=disponible, 1=no disponible)
total_bases: Número de bases de la estación
dock_bikes: Número de bicicletas ancladas
free_bases: Número de bases libres
reservations_count: Número de reservas activas


EN:
id: Base Station Code
latitude: Latitude of the station in WGS84 format
longitude: Station Longitude in WGS84 format
name: Station Name
light: Occupancy Degree (0=low, 1=medium, 2=high, 3=inactive)
number: Logical name of the Base Station
activate: Station active (0=Not active, 1=active)
no_available: Station Availability (0=available, 1=not available)
total_bases: Number of bases in the station
dock_bikes: Number of docked bikes
free_bases: Number of free bases
reservations_count: Number of active reservations
'''

'\nSP:\nDESCRIPCIÓN DEL OBJETO ESTACION\n\nid: Código de la Estación Base\nlatitude: Latitud de la estación en formato WGS84\nlongitude: Longitud de la Estación en formato WGS84\nname: Nombre de la Estación\nlight: Grado de Ocupación (0=baja, 1=media, 2=alta, 3=inactiva)\nnumber: Denominación lógica de la Estación Base\nactívate: Estación activa (0=No activa, 1=activa)\nno_available: Disponibilidad de la Estación (0=disponible, 1=no disponible)\ntotal_bases: Número de bases de la estación\ndock_bikes: Número de bicicletas ancladas\nfree_bases: Número de bases libres\nreservations_count: Número de reservas activas\n\n\nEN:\nid: Base Station Code\nlatitude: Latitude of the station in WGS84 format\nlongitude: Station Longitude in WGS84 format\nname: Station Name\nlight: Occupancy Degree (0=low, 1=medium, 2=high, 3=inactive)\nnumber: Logical name of the Base Station\nactivate: Station active (0=Not active, 1=active)\nno_available: Station Availability (0=available, 1=not available)\ntotal_

In [5]:
# SP: Saco las keys del archivo csv para poder eliminar varias columnas que no me interesan.

# EN: I remove the keys from the csv file so I can eliminate several columns that do not interest me.

bicimad = bicimad.drop(columns=['unnamed:_0', 'light', 'number'], axis=1)

bicimad.keys()

Index(['id', 'name', 'address', 'activate', 'no_available', 'total_bases',
       'dock_bikes', 'free_bases', 'reservations_count', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [6]:
# SP: Crear una nueva columna 'latitud' con los valores de latitud
# EN: Create a new 'latitud' column with the latitud values
bicimad['latitud'] = bicimad['geometry.coordinates'].apply(lambda x: x.strip('[]').split(', ')[1])
 
# SP: Crear una nueva columna 'longitud' con los valores de longitud
# EN: Create a new column 'longitud' with the longitud values.
bicimad['longitud'] = bicimad['geometry.coordinates'].apply(lambda x: x.strip('[]').split(', ')[0])

In [7]:
# SP: Borro la columna geometri.coodinates porque ya he creado dos columnas de ella
# EN: I delete the geometri.coodinates column because I have already created two columns from it

bicimad = bicimad.drop(columns=['geometry.coordinates'], axis=1)

In [8]:
bicimad

,id,name,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,latitud,longitud
0,1,1a - Puerta del Sol A,Puerta del Sol nº 1,1,1,30,0,0,0,Point,40.4172137,-3.7018341
1,2,1b - Puerta del Sol B,Puerta del Sol nº 1,1,1,30,0,0,0,Point,40.41731271011562,-3.701602938060457
2,3,2 - Miguel Moya,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,40.4205886,-3.7058415
3,4,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,40.4302937,-3.7069171
4,5,4 - Malasaña,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,40.4285524,-3.7025875
5,6,5 - Fuencarral,Calle Fuencarral nº 108,1,0,27,25,0,0,Point,40.42852,-3.70205
6,7,6 - Colegio Arquitectos,Calle Hortaleza nº 63,1,1,24,0,0,0,Point,40.424148,-3.698447
7,8,7 - Hortaleza,Calle Hortaleza nº 75,1,1,21,0,0,0,Point,40.4251906,-3.6977715
8,9,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5,1,0,24,22,0,0,Point,40.4278682,-3.6954403
9,10,9 - Plaza de San Miguel,Plaza de San Miguel nº 9,1,0,24,10,10,0,Point,40.4156057,-3.7095084


In [9]:
# SP: Convierto mi csv en un DataFrame
# EN: I convert my csv into a DataFrame

df_bicimad = pd.DataFrame(bicimad)

df_bicimad.head()

,id,name,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,latitud,longitud
0,1,1a - Puerta del Sol A,Puerta del Sol nº 1,1,1,30,0,0,0,Point,40.4172137,-3.7018341
1,2,1b - Puerta del Sol B,Puerta del Sol nº 1,1,1,30,0,0,0,Point,40.41731271011562,-3.701602938060457
2,3,2 - Miguel Moya,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,40.4205886,-3.7058415
3,4,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,40.4302937,-3.7069171
4,5,4 - Malasaña,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,40.4285524,-3.7025875


In [10]:
# SP: LLamamos a la API y sacamos archivo json
# EN: We call the API and get the json file

'''
API REST. We will use the API REST from the Portal de datos abiertos del Ayuntamiento de Madrid, 
where you can find the Catálogo de datos with more than 70 datasets. The API endpoint is https://datos.madrid.es/egob.

Mario: /catalogo/200215-0-instalaciones-deportivas.json
    
'''

api_endpoint_base = 'https://datos.madrid.es/egob'

my_endpoint = '/catalogo/200215-0-instalaciones-deportivas.json'

url = 'https://datos.madrid.es/egob/catalogo/200215-0-instalaciones-deportivas.json'

res = requests.get(url).json()

res

{'@context': {'c': 'http://www.w3.org/2002/12/cal#',
  'dcterms': 'http://purl.org/dc/terms/',
  'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
  'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
  'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
  'vcard': 'http://www.w3.org/2006/vcard/ns#',
  'schema': 'https://schema.org/',
  'title': 'vcard:fn',
  'id': 'dcterms:identifier',
  'relation': 'dcterms:relation',
  'references': 'dcterms:references',
  'address': 'vcard:adr',
  'area': 'loc:barrio',
  'district': 'loc:distrito',
  'locality': 'vcard:locality',
  'postal-code': 'vcard:postal-code',
  'street-address': 'vcard:street-address',
  'location': 'vcard:geo',
  'latitude': 'geo:lat',
  'longitude': 'geo:long',
  'organization': 'vcard:org',
  'organization-desc': 'dcterms:description',
  'accesibility': 'org:accesibilidad',
  'services': 'org:servicios',
  'schedule': 'org:horario',
  'organization-name': 'vcard:organization-name',
  'description': '

In [11]:
# SP: Comprobamos las keys existente
# EN: We check the existing keys

res.keys()

dict_keys(['@context', '@graph'])

In [12]:
# SP: Comprobamos los datos que nos dan las keys principales
# EN: We check the data that the main keys give us

res['@context'].keys()

dict_keys(['c', 'dcterms', 'geo', 'loc', 'org', 'vcard', 'schema', 'title', 'id', 'relation', 'references', 'address', 'area', 'district', 'locality', 'postal-code', 'street-address', 'location', 'latitude', 'longitude', 'organization', 'organization-desc', 'accesibility', 'services', 'schedule', 'organization-name', 'description', 'link', 'uid', 'dtstart', 'dtend', 'time', 'excluded-days', 'event-location', 'free', 'price', 'recurrence', 'days', 'frequency', 'interval', 'audience'])

In [13]:
res['@context']

{'c': 'http://www.w3.org/2002/12/cal#',
 'dcterms': 'http://purl.org/dc/terms/',
 'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
 'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
 'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
 'vcard': 'http://www.w3.org/2006/vcard/ns#',
 'schema': 'https://schema.org/',
 'title': 'vcard:fn',
 'id': 'dcterms:identifier',
 'relation': 'dcterms:relation',
 'references': 'dcterms:references',
 'address': 'vcard:adr',
 'area': 'loc:barrio',
 'district': 'loc:distrito',
 'locality': 'vcard:locality',
 'postal-code': 'vcard:postal-code',
 'street-address': 'vcard:street-address',
 'location': 'vcard:geo',
 'latitude': 'geo:lat',
 'longitude': 'geo:long',
 'organization': 'vcard:org',
 'organization-desc': 'dcterms:description',
 'accesibility': 'org:accesibilidad',
 'services': 'org:servicios',
 'schedule': 'org:horario',
 'organization-name': 'vcard:organization-name',
 'description': 'c:summary',
 'link': 'c:url',
 'uid': '

In [14]:
res['@graph']

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/11353987-instalacion-deportiva-basica-embalse-san-juan.json',
  '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosEspaciosDeporte/InstalacionesDeportivasBasicas',
  'id': '11353987',
  'title': 'Instalación Deportiva  Básica Embalse de San Juan',
  'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=fa29f533a3468710VgnVCM1000001d4a900aRCRD',
  'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/VillaDeVallecas'},
   'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/VillaDeVallecas/Barrio/EnsancheVallecas'},
   'locality': 'MADRID',
   'postal-code': '28051',
   'street-address': 'CALLE EMBALSE DE SAN JUAN 1'},
  'location': {'latitude': 40.36264285659688,
   'longitude': -3.5981484314802064},
  'organization': {'organization-desc'

In [15]:
len(res['@graph'])

506

In [16]:
res['@graph'][0].keys()

dict_keys(['@id', '@type', 'id', 'title', 'relation', 'address', 'location', 'organization'])

In [17]:
# SP: Los datos que necesitamos para poder trabajar son de la key @graph, 
# hago entonces un bucle para guardar todos los datos contenidos en @graph en la lista instalaciones

# EN: The data we need to be able to work is from the @graph key,
# I then make a loop to save all the data contained in @graph in the facilities list

instalaciones = []

for org in res['@graph']:
    
    instalaciones.append(org)

instalaciones

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/11353987-instalacion-deportiva-basica-embalse-san-juan.json',
  '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosEspaciosDeporte/InstalacionesDeportivasBasicas',
  'id': '11353987',
  'title': 'Instalación Deportiva  Básica Embalse de San Juan',
  'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=fa29f533a3468710VgnVCM1000001d4a900aRCRD',
  'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/VillaDeVallecas'},
   'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/VillaDeVallecas/Barrio/EnsancheVallecas'},
   'locality': 'MADRID',
   'postal-code': '28051',
   'street-address': 'CALLE EMBALSE DE SAN JUAN 1'},
  'location': {'latitude': 40.36264285659688,
   'longitude': -3.5981484314802064},
  'organization': {'organization-desc'

In [18]:
# SP: Miro sólo el primer dato para poder acotar mejor y ver que es lo que realmente necesito.
# EN: I only look at the first piece of information to be able to narrow it down better and see what I really need.

instalaciones[0]

{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/11353987-instalacion-deportiva-basica-embalse-san-juan.json',
 '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosEspaciosDeporte/InstalacionesDeportivasBasicas',
 'id': '11353987',
 'title': 'Instalación Deportiva  Básica Embalse de San Juan',
 'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=fa29f533a3468710VgnVCM1000001d4a900aRCRD',
 'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/VillaDeVallecas'},
  'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/VillaDeVallecas/Barrio/EnsancheVallecas'},
  'locality': 'MADRID',
  'postal-code': '28051',
  'street-address': 'CALLE EMBALSE DE SAN JUAN 1'},
 'location': {'latitude': 40.36264285659688, 'longitude': -3.5981484314802064},
 'organization': {'organization-desc': '\xa0\xa0Bus:

In [19]:
# SP: Selecciono los datos que creo necesarios y hago un bucle para guardar solo estos en otra lista datos.
# Al final decido suprimir organización

# EN: I select the data that I think is necessary and make a loop to save only this in another data list.
# In the end I decide to delete the organization

datos = []

for instalacion in instalaciones:
    
    titulo = instalacion['title']
    
    calle = instalacion['address']['street-address']
    
    #organizacion = instalacion['organization']['organization-desc'].strip('\xa0\xa0')
    
    latitud = instalacion['location']['latitude']
    
    longitud = instalacion['location']['longitude']
    
    datos.append((titulo, calle, latitud, longitud))

datos

[('Instalación Deportiva  Básica Embalse de San Juan',
  'CALLE EMBALSE DE SAN JUAN 1',
  40.36264285659688,
  -3.5981484314802064),
 ('Instalación Deportiva Básica Alto del Retiro',
  'CALLE ALTO DEL RETIRO 13 c/v calle Antonio Gades',
  40.36751469466709,
  -3.588264644913803),
 ('Instalación Deportiva Básica Avenida de Cerro Milano',
  'AVENIDA CERRO MILANO 1',
  40.36861899124304,
  -3.618356801446381),
 ('Instalación Deportiva Básica Avenida de las Suertes',
  'AVENIDA avenida de las Suertes ',
  40.370297266524084,
  -3.6120673755658),
 ('Instalación Deportiva Básica Avenida del Ensanche de Vallecas',
  'AVENIDA ENSANCHE DE VALLECAS 11',
  40.370297266524084,
  -3.6120673755658),
 ('Instalación Deportiva Básica Avenida del Mediterráneo',
  'INST. DEPORTIVA BASICA LUIS MITJANS ',
  40.40126476374574,
  -3.670479840720475),
 ('Instalación Deportiva Básica Camino de la Suerte',
  'CAMINO SUERTE 25',
  40.375813783817236,
  -3.617327043456161),
 ('Instalación Deportiva Básica Circuit

In [20]:
# SP: Creo un dataFrame con pandas con los datos que yo creo necesarios.
# EN: I create a dataFrame with pandas with the data that I think is necessary.

df_datos = pd.DataFrame(datos)

df_datos.columns = ('titulo', 'calle', 'latitud', 'longitud')

df_datos

,titulo,calle,latitud,longitud
0,Instalación Deportiva Básica Embalse de San Juan,CALLE EMBALSE DE SAN JUAN 1,40.362643,-3.598148
1,Instalación Deportiva Básica Alto del Retiro,CALLE ALTO DEL RETIRO 13 c/v calle Antonio Gades,40.367515,-3.588265
2,Instalación Deportiva Básica Avenida de Cerro ...,AVENIDA CERRO MILANO 1,40.368619,-3.618357
3,Instalación Deportiva Básica Avenida de las Su...,AVENIDA avenida de las Suertes,40.370297,-3.612067
4,Instalación Deportiva Básica Avenida del Ensan...,AVENIDA ENSANCHE DE VALLECAS 11,40.370297,-3.612067
5,Instalación Deportiva Básica Avenida del Medit...,INST. DEPORTIVA BASICA LUIS MITJANS,40.401265,-3.670480
6,Instalación Deportiva Básica Camino de la Suerte,CAMINO SUERTE 25,40.375814,-3.617327
7,Instalación Deportiva Básica Circuito de Autom...,CALLE NAVAMURES 2,40.372852,-3.626705
8,Instalación Deportiva Básica Colonia Vilda,CALLE PUENTELARRA 1 c/v Avda. Sta. Eugenia,40.381543,-3.610582
9,Instalación Deportiva Básica Eduardo Chillida,CALLE EDUARDO CHILLIDA 1,40.363022,-3.585074


In [21]:
# SP: Ahora tengo dos tablas diferentes BICIMAD y DF_DATOS que tienen los datos acotados con lo que necesito.
# EN: Now I have two different tables BICIMAD and DF_DATOS that have the data bounded to what I need.

df_datos  # SP: Aquí tenemos los datos de todos los centros deportivos. EN: Here we have the data of all the sports centers.

,titulo,calle,latitud,longitud
0,Instalación Deportiva Básica Embalse de San Juan,CALLE EMBALSE DE SAN JUAN 1,40.362643,-3.598148
1,Instalación Deportiva Básica Alto del Retiro,CALLE ALTO DEL RETIRO 13 c/v calle Antonio Gades,40.367515,-3.588265
2,Instalación Deportiva Básica Avenida de Cerro ...,AVENIDA CERRO MILANO 1,40.368619,-3.618357
3,Instalación Deportiva Básica Avenida de las Su...,AVENIDA avenida de las Suertes,40.370297,-3.612067
4,Instalación Deportiva Básica Avenida del Ensan...,AVENIDA ENSANCHE DE VALLECAS 11,40.370297,-3.612067
5,Instalación Deportiva Básica Avenida del Medit...,INST. DEPORTIVA BASICA LUIS MITJANS,40.401265,-3.670480
6,Instalación Deportiva Básica Camino de la Suerte,CAMINO SUERTE 25,40.375814,-3.617327
7,Instalación Deportiva Básica Circuito de Autom...,CALLE NAVAMURES 2,40.372852,-3.626705
8,Instalación Deportiva Básica Colonia Vilda,CALLE PUENTELARRA 1 c/v Avda. Sta. Eugenia,40.381543,-3.610582
9,Instalación Deportiva Básica Eduardo Chillida,CALLE EDUARDO CHILLIDA 1,40.363022,-3.585074


In [22]:
df_bicimad  # SP: Aqui tenemos los datos de todos los bicimad de madrid. EN: Here we have the data of all the BICIMAD in Madrid.

,id,name,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,latitud,longitud
0,1,1a - Puerta del Sol A,Puerta del Sol nº 1,1,1,30,0,0,0,Point,40.4172137,-3.7018341
1,2,1b - Puerta del Sol B,Puerta del Sol nº 1,1,1,30,0,0,0,Point,40.41731271011562,-3.701602938060457
2,3,2 - Miguel Moya,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,40.4205886,-3.7058415
3,4,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,40.4302937,-3.7069171
4,5,4 - Malasaña,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,40.4285524,-3.7025875
5,6,5 - Fuencarral,Calle Fuencarral nº 108,1,0,27,25,0,0,Point,40.42852,-3.70205
6,7,6 - Colegio Arquitectos,Calle Hortaleza nº 63,1,1,24,0,0,0,Point,40.424148,-3.698447
7,8,7 - Hortaleza,Calle Hortaleza nº 75,1,1,21,0,0,0,Point,40.4251906,-3.6977715
8,9,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5,1,0,24,22,0,0,Point,40.4278682,-3.6954403
9,10,9 - Plaza de San Miguel,Plaza de San Miguel nº 9,1,0,24,10,10,0,Point,40.4156057,-3.7095084


# Ahora debo calcular la distancia entre Centro deportivo y la parada de bicimad más cercana.

In [23]:
# SP: Cojo estas dos funciones que me dan en geo_calculations.py para calcular las distancias 
# entre dos puntos de inicio y final, pero no me funciona y siempre me dan un error que no puedo solucionar.

# EN: I take these two functions that they give me in geo_calculations.py to calculate the distances
# between two start and end points, but it doesn't work for me and they always give me an error that I can't solve.

'''
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    
    start = to_mercator(lat_start, long_start)
    
    finish = to_mercator(lat_finish, long_finish)
    
    return start.distance(finish)

'''

'\ndef to_mercator(lat, long):\n    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres\n    c = gpd.GeoSeries([Point(lat, long)], crs=4326)\n    c = c.to_crs(3857)\n    return c\n\ndef distance_meters(lat_start, long_start, lat_finish, long_finish):\n    \n    # return the distance in metres between to latitude/longitude pair points in degrees \n    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)\n    \n    start = to_mercator(lat_start, long_start)\n    \n    finish = to_mercator(lat_finish, long_finish)\n    \n    return start.distance(finish)\n\n'

In [24]:
# SP: Decido usar la formula Haversine porque las funciones que se me dan en el ejercicio siempre me dan el error de 
# atributos ambiguos.
# Tengo que poner el radio de la tierra en metros porque esta en kilometros.

# EN: I decide to use the Haversine formula because the functions given to me in the exercise always give me the error
# ambiguous attributes.
# I have to put the radius of the earth in meters because it is in kilometers.

def haversine_meters(lat1, lon1, lat2, lon2):
    # SP: convierte los grados a radio. EN: convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371000  # Radio de la tierra en metros.
    return c * r

In [25]:
# SP: Necesito centro deportivo y su estacion bicimad mas cercana. 
# Necesito comparar la latitud y longitud de los centros deportivos con la latitud y longitud de las estaciones bicimad.
# Tengo que crear una tabla con el nombre del centro deportivo, su estación mas cercana y la distancia en metros.

# EN: I need a sports center and the nearest bike station.
# I need to compare the latitude and longitude of the sports centers with the latitude and longitude of the bike stations.
# I have to create a table with the name of the sports center, its closest station and the distance in meters

In [26]:
# SP: Función para encontrar los puntos más cercanos en df2 para cada punto en df1
# EN: Function to find the closest points in df2 for each point in df1

def encontrar_puntos_cercanos(df1, df2):
    
    paradas_cercanas = []
    
    for indice1, fila1 in df1.iterrows():
        
        min_distance = np.inf # Inicializo en el numero minimo positivo, con cero me da error
        #min_distance = 0
        
        indice_cercano = None
        
        for indice2, fila2 in df2.iterrows():
            
            distance = haversine_meters(fila1['latitud'], fila1['longitud'], fila2['latitud'], fila2['longitud'])
            
            if distance < min_distance:
                
                min_distance = distance
                
                indice_cercano = indice2
                
        paradas_cercanas.append(indice_cercano)
        
    return paradas_cercanas


In [27]:
# SP: Convertir columnas de latitud y longitud de los dos dataframes en float
# EN: Convert latitud and longitud columns in both dataframes to float

df_bicimad['latitud'] = df_bicimad['latitud'].astype(float)
df_bicimad['longitud'] = df_bicimad['longitud'].astype(float)
df_datos['latitud'] = df_datos['latitud'].astype(float)
df_datos['longitud'] = df_datos['longitud'].astype(float)

In [28]:
# SP: buscar las paradas cercanas
# EN: Find closest points

closest_indices = encontrar_puntos_cercanos( df_datos, df_bicimad)
df_datos['paradas_cercanas_df2'] = closest_indices

# SP: Unir los dos dataFrames
# EN: Merge the two dataframes

df_centroxbicimad = pd.merge( df_datos, df_bicimad, left_on='paradas_cercanas_df2', right_index=True, suffixes=('_df1', '_df2'))


In [29]:
# SP: Ahora, usando esta función para calcular distancias en el DataFrame fusionado
# EN: Now, using this function to calculate distances in the merged DataFrame

df_centroxbicimad['metros_distancia'] = df_centroxbicimad.apply(
    lambda row: haversine_meters(row['latitud_df1'], row['longitud_df1'], row['latitud_df2'], row['longitud_df2']),
    axis=1
)

In [30]:
df_centroxbicimad.columns

Index(['titulo', 'calle', 'latitud_df1', 'longitud_df1',
       'paradas_cercanas_df2', 'id', 'name', 'address', 'activate',
       'no_available', 'total_bases', 'dock_bikes', 'free_bases',
       'reservations_count', 'geometry.type', 'latitud_df2', 'longitud_df2',
       'metros_distancia'],
      dtype='object')

In [34]:
# SP: visualizo el nuevo dataframe con las columnas que yo decido. Por eso antes veo que columnas tengo.
# EN: I display the new dataframe with the columns that I decide. That's why first I see what columns I have.

df_centroxbicimad[['titulo', 'calle', 'latitud_df1', 'longitud_df1', 'metros_distancia', 'name', 'address', 'no_available', 'latitud_df2', 'longitud_df2']]

,titulo,calle,latitud_df1,longitud_df1,metros_distancia,name,address,no_available,latitud_df2,longitud_df2
0,Instalación Deportiva Básica Embalse de San Juan,CALLE EMBALSE DE SAN JUAN 1,40.362643,-3.598148,5207.956221,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
1,Instalación Deportiva Básica Alto del Retiro,CALLE ALTO DEL RETIRO 13 c/v calle Antonio Gades,40.367515,-3.588265,5360.872027,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
2,Instalación Deportiva Básica Avenida de Cerro ...,AVENIDA CERRO MILANO 1,40.368619,-3.618357,3788.529601,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
3,Instalación Deportiva Básica Avenida de las Su...,AVENIDA avenida de las Suertes,40.370297,-3.612067,3849.472053,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
4,Instalación Deportiva Básica Avenida del Ensan...,AVENIDA ENSANCHE DE VALLECAS 11,40.370297,-3.612067,3849.472053,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
6,Instalación Deportiva Básica Camino de la Suerte,CAMINO SUERTE 25,40.375814,-3.617327,3097.010243,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
7,Instalación Deportiva Básica Circuito de Autom...,CALLE NAVAMURES 2,40.372852,-3.626705,3131.641857,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
8,Instalación Deportiva Básica Colonia Vilda,CALLE PUENTELARRA 1 c/v Avda. Sta. Eugenia,40.381543,-3.610582,2917.987728,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
9,Instalación Deportiva Básica Eduardo Chillida,CALLE EDUARDO CHILLIDA 1,40.363022,-3.585074,5901.935250,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587
12,Instalación Deportiva Básica La Unión,CALLE ENRIQUE GARCIA ALVAREZ 58 c/v Diego Oliv...,40.383449,-3.614864,2515.855828,221 - Pavones,Calle Hacienda de Pavones nº 342,0,40.400368,-3.634587


In [32]:
# SP: la longitud coincide con la tabla de centros deportivos

# EN: the length matches the table of sports centers

len(df_centroxbicimad.index)

506

In [33]:
# SP: Exporto la tabla creada de la union de los dos DataFrames.

# EN: Exporto la tabla creada de la unión de los dos DataFrames.

df_centroxbicimad.to_csv('data/centroxbicimad.csv', index=False)
